<a href="https://colab.research.google.com/github/vincimech010233/MachineLearningBasics/blob/master/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers torch torchvision

In [2]:
import gradio as gr
from transformers import pipeline
from torchvision import models, transforms
from PIL import Image
import torch

In [ ]:
# Cargar un modelo preentrenado de Hugging Face para el análisis de sentimiento
sentiment_analysis = pipeline("sentiment-analysis")

# Cargar un modelo preentrenado de torchvision para clasificación de imágenes
image_model = models.resnet18(pretrained=True)
image_model.eval()

In [4]:
# Definir la transformación de la imagen para el modelo de torchvision
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Define la función que utilizará ambos modelos para hacer inferencia
def my_multimodal_model(text_input, image_input):
    # Procesar texto y realizar análisis de sentimiento
    sentiment_result = sentiment_analysis(text_input)

    # Procesar imagen y realizar clasificación de imagen
    image = Image.fromarray(image_input.astype('uint8'), 'RGB')
    image = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        image_prediction = image_model(image)
    top5_prob, top5_catid = torch.topk(torch.nn.functional.softmax(image_prediction[0], dim=0), 5)
    imagenet_index = {idx: 'Clase ' + str(idx) for idx in range(1000)}
    image_result = ", ".join([imagenet_index[catid] for catid in top5_catid.numpy()])

    return sentiment_result, image_result


In [7]:
# Define la interfaz de Gradio
iface = gr.Interface(
    fn=my_multimodal_model,
    inputs=[
        gr.Textbox(lines=2, placeholder="Escribe aquí tu texto..."),
        gr.Image()
    ],
    outputs=[
        gr.Text(label="Texto generado"),
        gr.Label(label="Clasificación de imagen")
    ]
)

In [8]:
# Lanzar la interfaz
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0e851ec15a305a72bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
